In [1]:
import numpy as np

#Deterministic environment for optimal maintenance problem
#States: {0,1} Healthy, Faulty
#Actions: {0,1} Do nothing, Repair
#Rewards: -1,0,-10 
p = np.zeros((2,2,2)).astype(int)
#p[s,a]=(r,s')
p[0,0]=[0,0]
p[0,1]=[-10,0]
p[1,0]=[-1,1]
p[1,1]=[-10,0]

def epsilon_greedy_pi(Q,state,epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(2)
    else:
        return Q[state,:].argmax()

def take_action(state_action):
    return p[state_action]

def DynaQ(n, n_timestep, epsilon, alpha, gamma):
    #Initialize action-value function and model
    Q = np.zeros((2,2)).astype(int)
    model = np.zeros((p.shape)).astype(int)
    observed_s_a = {}
    
    for i in range(n_timestep):
        state = np.random.randint(2)
        action = epsilon_greedy_pi(Q,state,epsilon)

        #Experience
        reward, next_state = take_action((state,action))
        if state in observed_s_a:
            observed_s_a[state].append(action)
        else:
            observed_s_a[state] = [action]
        
        #Direct RL
        Q[state,action] += alpha * (reward + gamma * np.argmax(Q[next_state,:]) - Q[state,action])
        
        #Model learning
        model[state,action] = [reward,next_state]
        
        #Planning
        for j in range(n):
            state = np.random.choice(list(observed_s_a.keys()))
            action = np.random.choice(observed_s_a[state])
            
            reward, next_state = model[state,action]
            Q[state,action] += alpha * (reward + gamma * np.argmax(Q[next_state,:]) - Q[state,action])
    
    return Q, model

In [2]:
Q, model = DynaQ(n=5, n_timestep=100, epsilon=0.1, alpha=0.1, gamma=0.9)

In [3]:
model

array([[[  0,   0],
        [-10,   0]],

       [[ -1,   1],
        [-10,   0]]])

In [5]:
p == model

array([[[ True,  True],
        [ True,  True]],

       [[ True,  True],
        [ True,  True]]])